In [1]:
!pip install transformers torch scikit-learn pandas


In [2]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.nn.functional import softmax
from transformers import logging

logging.set_verbosity_error()

model_name = 'nlptown/bert-base-multilingual-uncased-sentiment'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

csv_path = 'reviews.csv'
df = pd.read_csv(csv_path)

# Define function for sentiment analysis
def analyze_sentiment(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    probs = softmax(outputs.logits, dim=1)
    score = torch.argmax(probs).item() + 1  # 1 to 5 scale
    confidence = torch.max(probs).item()

    if score >= 4:
        sentiment = "Positive"
    elif score == 3:
        sentiment = "Neutral"
    else:
        sentiment = "Negative"
    
    return sentiment, score, round(confidence, 2)

df[['Sentiment', 'Predicted Star Rating', 'Confidence']] = df['content'].apply(
    lambda x: pd.Series(analyze_sentiment(str(x)))
)

print(df[['content', 'Sentiment', 'Predicted Star Rating', 'Confidence']])

df.to_csv('reviews_with_sentiment.csv', index=False)


                                                content Sentiment  \
0                         I cannot open the app anymore  Negative   
1     I have been begging for a refund from this app...  Negative   
2     Very costly for the premium version (approx In...  Negative   
3     Used to keep me organized, but all the 2020 UP...  Negative   
4                                   Dan Birthday Oct 28  Positive   
...                                                 ...       ...   
1995  They double billed me for my premium subscript...  Negative   
1996                   This is the beat app of all !!!!  Positive   
1997  I downloaded this app after hearing good thing...  Negative   
1998  It just vibrate I never get an alert. It only ...  Negative   
1999                                 Problem with login  Negative   

      Predicted Star Rating  Confidence  
0                         1        0.65  
1                         1        0.82  
2                         2        0.51  
3  